<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-модулей-и--библиотек" data-toc-modified-id="Импорт-модулей-и--библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт модулей и  библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт модулей и  библиотек

In [1]:
import pandas as pd
import numpy as np
import math 

from IPython.display import display
from tqdm.notebook import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import time
from datetime import timedelta
import matplotlib.pyplot as plt


import re
import string
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 


from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, cross_val_score, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier
from sklearn.utils import shuffle


import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Загрузка данных

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
display(df.info())
df['toxic'].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

0    0.898321
1    0.101679
Name: toxic, dtype: float64

В данных 2 стобца, 159571 строк, безпропусков, целевой признак имеет явный дисбаланс классов, учтем при построении моделей

###  Лемматизация

In [3]:
def clear_text(text):
    return ' '.join((re.sub(r'[^a-zA-Z]', ' ', text)).lower().split())

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [4]:
def lemm_text(text):
    text_clear = clear_text(text)
    text_token = word_tokenize(text_clear)
    text_token_without_stop_words = [
        word for word in text_token if word not in stop_words]
    lemma = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text_token_without_stop_words]
    
    return " ".join(lemma)

In [5]:
df['lemm_text'] = df['text'].progress_apply(lambda x: lemm_text(x))

## Обучение

### Разделение на выборки

In [6]:
df_train, test = train_test_split(df, test_size=0.2, random_state=13, stratify=df['toxic'])
train, valid = train_test_split(df_train, test_size=0.25, random_state=13, stratify=df_train['toxic'])

target_train = train['toxic']
target_valid = valid['toxic']
target_test = test['toxic']

### TF-IDF

In [7]:
corpus = train['lemm_text'].values.astype('U')

count_tf_idf = TfidfVectorizer() 
features_train = count_tf_idf.fit_transform(corpus)

features_valid = count_tf_idf.transform(valid['lemm_text'].values.astype('U')) 
features_test = count_tf_idf.transform(test['lemm_text'].values.astype('U')) 

print("Размер матрицы обучающей выборки:", features_train.shape)
print("Размер матрицы валидационной выборки:", features_valid.shape)
print("Размер матрицы тестовой выборки:", features_test.shape)

Размер матрицы обучающей выборки: (95742, 111654)
Размер матрицы валидационной выборки: (31914, 111654)
Размер матрицы тестовой выборки: (31915, 111654)


### LogisticRegression

In [8]:
%%time
lr = LogisticRegressionCV(cv=3, random_state=12345, class_weight='balanced')
lr.fit(features_train, target_train)


CPU times: user 6min 1s, sys: 7min 27s, total: 13min 28s
Wall time: 13min 28s


LogisticRegressionCV(Cs=10, class_weight='balanced', cv=3, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=12345, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [9]:
def pred(model, features, target):
    prediction = model.predict(features)
    f1 = f1_score(target, prediction)
    print('F1-мера: {:.3f}'.format(f1))
    return f1

In [10]:
pred(lr, features_valid, target_valid)
pred(lr,features_test,target_test)

F1-мера: 0.764
F1-мера: 0.755


0.7547169811320755

Логистическая регрессия справилась с задачей метрика  F1 =  0.755 на тестовой выборке

### RandomForestClassifier

Гридсерч выдал лучшие параметры:
                             random_state=12345, 
                             class_weight='balanced',
                             max_depth=11, 
                             min_samples_leaf=1, 
                             min_samples_split=4,
                             n_estimators=40

In [12]:
%%time
rfc = RandomForestClassifier(random_state=12345, 
                             class_weight='balanced',
                             max_depth=11, 
                             min_samples_leaf=1, 
                             min_samples_split=4,
                             n_estimators=40)
rfc.fit(features_train, target_train)


CPU times: user 1.32 s, sys: 0 ns, total: 1.32 s
Wall time: 1.32 s


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=11, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=4, min_weight_fraction_leaf=0.0,
                       n_estimators=40, n_jobs=None, oob_score=False,
                       random_state=12345, verbose=0, warm_start=False)

In [13]:
pred(rfc, features_valid, target_valid)
pred(rfc,features_test,target_test)

F1-мера: 0.399
F1-мера: 0.392


0.3922877271041898

Рандомный лес с задачей не справился

## Выводы

С задачей справилась модель Логистическрой регресси метрика F1=0,75 , Рандомный лес не справился с задаче, показав очень низкий результат 0.392. 

Для ревьюера: пытался использовать  BERT, в итоге отказался от этой идеии т.к. мой старичек(мак) не тянет, также пытался использовать катбусткласификатор, но он просто убивает кернел(, мне кажется градиетныйбустинг лучше бы справился с задачей по качеству метрики